In [19]:
#Load all necessary packages
library(ggplot2)
library(dplyr)
library(gridExtra)
library(data.table)
library(rvest)
library(xml2)
library(tidyverse)
library(stringr)

ERROR: Error: package or namespace load failed for 'tidyverse' in loadNamespace(j <- i[[1L]], c(lib.loc, .libPaths()), versionCheck = vI[[j]]):
 namespace 'tibble' 2.1.1 is already loaded, but >= 3.0.0 is required


In [20]:
#To scrap info on postal codes, boroughs and neighbourhoods in Toronto, Canada from a url
data <- read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
body_nodes <- data %>% html_node('tbody') %>% html_children()

#extract information from node <tr> for list of postal codes, boroughs and neighbourhoods
list <- data %>% rvest::html_nodes('tbody') %>%
  xml2::xml_find_all("//tr") %>% rvest::html_text()

In [21]:
#To tidy up vectors
list <- list[2:185] #to remove the first row in the vector
postal_codes <- substr(list, 1, 3) #To capture only the postal codes
boroughs <- sub(".*\n\n *(.*?) *\n\n.*", "\\1", list) #to capture list of boroughs using regex.
neighbourhoods <- sub(".*\n\n *(.*?) *\n*", "\\2", list) #to capture list of neighbourhoods using regex.

In [29]:
#to create one dataframe
table_of_postal_codes <- data.table(Postal_Code = postal_codes, Borough=boroughs, Neighbourhood=neighbourhoods, stringsAsFactors = FALSE)

#to remove all rows with Not assigned in column Borough
table_of_postal_codes <- table_of_postal_codes[1:180]
table_of_postal_codes <- table_of_postal_codes[!(table_of_postal_codes$Borough == 'Not assigned')]

#to find the shape of the dataframe
str(table_of_postal_codes)


Classes 'data.table' and 'data.frame':	103 obs. of  3 variables:
 $ Postal_Code  : chr  "M3A" "M4A" "M5A" "M6A" ...
 $ Borough      : chr  "North York" "North York" "Downtown Toronto" "North York" ...
 $ Neighbourhood: chr  "Parkwoods\n" "Victoria Village\n" "Regent Park, Harbourfront\n" "Lawrence Manor, Lawrence Heights\n" ...
 - attr(*, ".internal.selfref")=<externalptr> 


In [30]:
#to see the summary of the dataframe
summary(table_of_postal_codes)

 Postal_Code          Borough          Neighbourhood     
 Length:103         Length:103         Length:103        
 Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character  

In [32]:
#to view the first ten rows of the dataframe
head(table_of_postal_codes, 10)

Postal_Code,Borough,Neighbourhood
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
M1B,Scarborough,"Malvern, Rouge"
M3B,North York,Don Mills
M4B,East York,"Parkview Hill, Woodbine Gardens"
M5B,Downtown Toronto,"Garden District, Ryerson"


In [33]:
#to view the last ten rows of the dataframe
tail(table_of_postal_codes, 10)

Postal_Code,Borough,Neighbourhood
M8W,Etobicoke,"Alderwood, Long Branch"
M9W,Etobicoke,"Northwest, West Humber - Clairville"
M1X,Scarborough,Upper Rouge
M4X,Downtown Toronto,"St. James Town, Cabbagetown"
M5X,Downtown Toronto,"First Canadian Place, Underground city"
M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
M4Y,Downtown Toronto,Church and Wellesley
M7Y,East Toronto,"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto"
M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East"
M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West"
